<a href="https://colab.research.google.com/github/mettinger/eegCompress/blob/main/predictiveCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mne neptune neptune_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import random

import torch
from torch.utils.data.dataset import Dataset

import neptune
from neptune_pytorch import NeptuneLogger

import datetime
import pytz
timeZone = pytz.timezone('America/Los_Angeles')


In [ ]:
def imageCompare(start, nBlock, channel = 0, plotOption="both"):

    original = np.array([]).astype('float32')
    decoded = np.array([]).astype('float32')

    for i in range(nBlock):
      original = np.append(original, data[channel,start:start + numSampleInput])

      modelInput = np.reshape(data[:, start:start + numSampleInput], (inSize, -1), order='F').astype('float32').flatten()
      encoded = model.encoder(torch.tensor(modelInput))
      decoded = np.append(decoded, np.reshape(model.decoder(encoded).detach().numpy(), (nChannel, numSampleInput),order="F")[channel, :])

    fig = plt.figure()
    if plotOption == "both":
        plt.plot(original, label='original')
        plt.plot(decoded, label='decoded')
        plt.ylim([-.5,.5])
        plt.legend()
    elif plotOption == "orig":
        plt.plot(original)
        plt.ylim([-.5,.5])
        plt.title('original')
    else:
        plt.plot(decoded)
        plt.ylim([-.5,.5])
        plt.title('decoded')

    return fig, encoded, decoded



def sizeToLayerList(encoderSizeList,
					decoderSizeList,
					encoderActivationList,
					decoderActivationList):
	encoderLayerList = []
	decoderLayerList = []

	for i in range(0, len(encoderSizeList) - 1):
		thisLayer = torch.nn.Linear(encoderSizeList[i], encoderSizeList[i + 1])
		torch.nn.init.xavier_uniform_(thisLayer.weight)

		encoderLayerList.append(thisLayer)
		if encoderActivationList[i]:
			encoderLayerList.append(torch.nn.ReLU())

	#decoderSizeList = [encoderSizeList[-2]] + decoderSizeList
	for i in range(0, len(decoderSizeList) - 1):
		thisLayer = torch.nn.Linear(decoderSizeList[i], decoderSizeList[i + 1])
		torch.nn.init.xavier_uniform_(thisLayer.weight)

		decoderLayerList.append(thisLayer)
		if decoderActivationList[i]:
			decoderLayerList.append(torch.nn.ReLU())

	return encoderLayerList, decoderLayerList



class AE(torch.nn.Module):
	def __init__(self, encoderSizeList, decoderSizeList, encoderActivationList, decoderActivationList):
		super().__init__()

		encoderLayerList, decoderLayerList = sizeToLayerList(encoderSizeList,
															decoderSizeList,
															encoderActivationList,
															decoderActivationList)

		self.encoder = torch.nn.Sequential(*encoderLayerList)
		self.decoder = torch.nn.Sequential(*decoderLayerList)

	def forward(self, x):
		encoded = self.encoder(x)
		decoded = self.decoder(encoded)
		return decoded



class CustomDataset(Dataset):
  def __init__(self, eegNumpy, numSampleInput):
    self.eegNumpy = eegNumpy
    self.numSampleInput = numSampleInput
    self.nChannel, self.nSample = eegNumpy.shape

  def __len__(self):
    return int(self.nSample/self.numSampleInput)
		#return self.nSample - self.numSampleInput

  def __getitem__(self, idx):
    image = np.reshape(self.eegNumpy[:,idx * numSampleInput : (idx + 1) * numSampleInput], (self.nChannel * self.numSampleInput,-1), order='F').transpose().astype('float32')
    #image = np.reshape(self.eegNumpy[:,idx:idx + self.numSampleInput], (self.nChannel * self.numSampleInput,-1), order='F').transpose().astype('float32')
    return image, 0



def loadModel(path, trainBool = True):

  checkpoint = torch.load(path, weights_only=True)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  loss = checkpoint['loss']

  model.eval()
  if trainBool:
    model.train()
  else:
    model.eval()

  return model, optimizer, epoch, loss


In [ ]:
data = np.load('/content/drive/MyDrive/NeuroResearch/Data/eegCompress/processedData/processedSVD001.npz')['arr_0']

### Define Model, etc.

In [ ]:
# Set in/out parameters
numSampleInput = 50
inSize = nChannel * numSampleInput

# Construct the DataLoader
dataset = CustomDataset(data, numSampleInput)
batch_size = 32
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
print("Batches per epoch: " + str(int(nSample/(numSampleInput * batch_size))) + "\n")


# Make the model
encoderRatioList = [1., 1.]
decoderRatioList = [1., 1.]
encoderSizeList = [int(inSize * i) for i in encoderRatioList]
decoderSizeList = [int(inSize * i) for i in decoderRatioList]
encoderActivationList = [True]
decoderActivationList = [False]

model = AE(encoderSizeList, decoderSizeList, encoderActivationList, decoderActivationList)
print(model)
loss_function = torch.nn.MSELoss()

Batches per epoch: 797

AE(
  (encoder): Sequential(
    (0): Linear(in_features=950, out_features=950, bias=True)
    (1): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=950, out_features=950, bias=True)
  )
)


In [ ]:
run = neptune.init_run(
    project="jettinger35/eegCompress",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzMjFlMzY2MS1iOWZiLTRmZWEtOGMwNy0zOTVkMTljOGVjYTMifQ==",
    #with_id="EEG-116"
    )

npt_logger = NeptuneLogger(
    run=run,
    model=model)

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/jettinger35/eegCompress/e/EEG-120


In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.0)
#optimizer = torch.optim.Adam(model.parameters())#, lr = 1e-1, weight_decay = 1e-8)

totalEpoch = 0

In [ ]:
epochs = 1000
numSamplePlot = 3
nBlockToPlot = 5
saveBool = 1

for epoch in range(epochs):
	for (image, _) in loader:
		# Output of Autoencoder
		reconstructed = model(image.to(torch.float32))
		# Calculating the loss function
		loss = loss_function(reconstructed, image)

		# The gradients are set to zero,
		# the gradient is computed and stored.
		# .step() performs parameter update
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		run[npt_logger.base_namespace]["train/log_loss"].append(np.log(loss.item()))

	for i in range(numSamplePlot):
		startPlot = random.randint(0, nSample - numSampleInput)
		fig, _, _ = imageCompare(startPlot, nBlockToPlot)
		plt.title("Epoch, Start, Blocks: " + str((totalEpoch, startPlot, nBlockToPlot)))
		run["fig"].append(fig)
		plt.close()

	totalEpoch += 1

if saveBool:
	saveName = 'savedModel_' + str(datetime.datetime.now().astimezone(timeZone)) + '.pt'
	torch.save({'epoch': epoch,
							'model_state_dict': model.state_dict(),
							'optimizer_state_dict': optimizer.state_dict(),
							'loss': loss}, '/content/drive/MyDrive/Colab Notebooks/' + saveName)
	print("Model has been saved: " + saveName)

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
plotBool = 0

if plotBool:
  startPlot = 0
  fig, encoded, decoded = imageCompare(startPlot)
  plt.show(fig)

In [ ]:
loadBool = 0

if loadBool:
  modelPath = '/content/drive/MyDrive/Colab Notebooks/savedModel_2025-01-21 08:35:06.519601-08:00.pt'
  model, optimizer, totalEpoch, loss = loadModel(modelPath)

### Misc

In [ ]:
'''
data = np.random.random((1,100000))
data = data - np.mean(data)
data = data/np.std(data)
nChannel = data.shape[0]
'''

'\ndata = np.random.random((1,100000))\ndata = data - np.mean(data)\ndata = data/np.std(data)\nnChannel = data.shape[0]\n'

In [ ]:
'''
list(model.parameters())[0].grad


for name, param in model.named_parameters():
    print((name, param))


optimizer.param_groups[0]['lr']=.0001
'''

"\nlist(model.parameters())[0].grad\n\n\nfor name, param in model.named_parameters():\n    print((name, param))\n\n\noptimizer.param_groups[0]['lr']=.0001\n"